In [10]:
import pandas as pd

In [11]:
odeme = pd.read_excel('ode.xlsx')

In [12]:
sant = pd.read_excel('santander.xlsx', sheet_name='bank')

In [13]:
odeme.head()

,COST OFFICE,PAY.DATE,PAYMENT TYPE,COMPANY/To whom,EXPLANATION 1,EXPLANATION 2 (POLSKI),AMOUNT,CURRENCY,BANK,BANK CODE,ACCOUNTING CODE,ACCOUNTING NAME,TITTLE OF PAYMENT,PAY.TYPE FOR PLAN
0,WAW,2024-10-30,RESERVATION OVER COMMISSION,"P.H.U. ""EM-KA"" Aleksandra Wolniak-Wiśniewska",COMMISSION FOR RES :1278394,NaN,8395.0,PLN,Santander Bank Polska,Santander Bank PLN (COMMISSIONS REFUNDS),NaN,NaN,FS 37/2024,AGENCY COMMISION
1,WAW,2024-10-30,RESERVATION OVER COMMISSION,TRAVELPLANET.PL SPÓŁKA AKCYJNA,COMMISSION FOR RES : 1265394,NaN,8125.0,PLN,Santander Bank Polska,Santander Bank PLN (COMMISSIONS REFUNDS),NaN,NaN,FSF/19/24/10/0002,AGENCY COMMISION
2,WAW,2024-10-30,RESERVATION OVER COMMISSION,"ANIELA CHOLEWA BIURO TURYSTYCZNE ""NELTRAVEL""",COMMISSION FOR RES :1361946,NaN,7385.0,PLN,Santander Bank Polska,Santander Bank PLN (COMMISSIONS REFUNDS),NaN,NaN,FV 159/2024,AGENCY COMMISION
3,WAW,2024-10-30,RESERVATION OVER COMMISSION,WAKACJE.PL SPÓŁKA AKCYJNA,COMMISSION FOR RES :1362138,NaN,7080.0,PLN,Santander Bank Polska,Santander Bank PLN (COMMISSIONS REFUNDS),NaN,NaN,08136/09/24/BP,AGENCY COMMISION
4,WAW,2024-10-30,RESERVATION OVER COMMISSION,WAKACJE.PL SPÓŁKA AKCYJNA,COMMISSION FOR RES :1423688,NaN,6965.0,PLN,Santander Bank Polska,Santander Bank PLN (COMMISSIONS REFUNDS),NaN,NaN,10351/09/24/BP,AGENCY COMMISION


In [14]:
sant.head()

,Firma,Rachunek,Data księgowania,Data przyjęcia do realizacji,Strona transakcji,Rachunek strony transakcji,Tytuł,Kwota,Waluta,Typ operacji
0,OREX TRAVEL SP Z O O,1.110902e+25,2024-10-30,2024-10-30,BIURO PODROZY JAWORKO,4.310204e+25,ZWROT Z REZERWACJI 1356311,-121.54,PLN,PRZELEW ELIXIR - IBIZNES24
1,OREX TRAVEL SP Z O O,1.110902e+25,2024-10-30,2024-10-30,BIURO PODROZY JAWORKO,4.310204e+25,ZWROT Z REZERWACJI 1356869,-126.22,PLN,PRZELEW ELIXIR - IBIZNES24
2,OREX TRAVEL SP Z O O,1.110902e+25,2024-10-30,2024-10-30,BIURO PODROZY WYMARZONE WAKACJE SPO LKA Z OGRA...,9.111402e+25,45572,-15315.00,PLN,PRZELEW ELIXIR - IBIZNES24
3,OREX TRAVEL SP Z O O,1.110902e+25,2024-10-30,2024-10-30,BIURO PODROZY JAWORKO BIURO PODROZY JAWORKO S...,4.310204e+25,45566,-25110.00,PLN,PRZELEW ELIXIR - IBIZNES24
4,OREX TRAVEL SP Z O O,1.110902e+25,2024-10-30,2024-10-30,Biuro Podrozy Eva Travel Ewa Maka JOZEFA PILSU...,9.487920e+25,FV/3/2024/09,-16649.99,PLN,PRZELEW ELIXIR - IBIZNES24


In [15]:
sant['Kwota'] = -sant['Kwota']

In [16]:
# group odeme by "TITTLE OF PAYMENT" and make a list from the "AMOUNT" column, add a column with sum of all the "AMOUNT", drop all other columns
odeme_grouped = odeme.groupby('TITTLE OF PAYMENT')['AMOUNT'].apply(list).reset_index()
odeme_grouped['sum'] = odeme_grouped['AMOUNT'].apply(sum)

# group sant by "Tytuł" and make a list of the "Kwota" column, add a column with sum of all the "Kwota", drop all other columns
sant_grouped = sant.groupby('Tytuł')['Kwota'].apply(list).reset_index()
sant_grouped['sum'] = sant_grouped['Kwota'].apply(sum)

# merge the two dataframes on the "TITTLE OF PAYMENT" and "Tytuł" columns
merged = pd.merge(odeme_grouped, sant_grouped, left_on='TITTLE OF PAYMENT', right_on='Tytuł', how='outer')

In [17]:
# move exact matches to the "exact" dataframe
exact = merged[merged['Tytuł'] == merged['TITTLE OF PAYMENT']].copy()

# drop the exact matches from the merged dataframe
merged = merged[merged['Tytuł'] != merged['TITTLE OF PAYMENT']]

In [18]:
# save them both to 1 excel file, but 2 different sheets
with pd.ExcelWriter('output.xlsx') as writer:
    exact.to_excel(writer, sheet_name='exact', index=False)
    merged.to_excel(writer, sheet_name='merged', index=False)